In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/msmarcotop100/msmarco-doctrain-top100
/kaggle/input/msmarco-docs-data/msmarco-docs.tsv
/kaggle/input/ms-marco-queries/msmarco-doctrain-queries.tsv


In [3]:
pip install fast-autocomplete


You should consider upgrading via the '/opt/conda/bin/python3.7 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [1]:
pip install stanfordnlp

     |████████████████████████████████| 158 kB 413 kB/s eta 0:00:01
You should consider upgrading via the '/opt/conda/bin/python3.7 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [4]:
stanfordnlp.download('en')

NameError: name 'stanfordnlp' is not defined

In [ ]:
pip install lupyne

In [6]:
pip install editdistance

     |████████████████████████████████| 179 kB 418 kB/s eta 0:00:01
You should consider upgrading via the '/opt/conda/bin/python3.7 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [ ]:
pip install pylucene4

In [15]:
import requests
import random
import editdistance
import stanfordnlp as st
import spacy 

from tqdm import tqdm
tqdm.pandas()

/opt/conda/lib/python3.7/site-packages/tqdm/std.py:666: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import Panel


In [13]:
st.download('en')

Using the default treebank "en_ewt" for language "en".
Would you like to download the models for: en_ewt now? (Y/n)
Y

Default download directory: /root/stanfordnlp_resources
Hit enter to continue or type an alternate directory.


Download location: /root/stanfordnlp_resources/en_ewt_models.zip


100%|██████████| 235M/235M [00:33<00:00, 7.11MB/s] 



Download complete.  Models saved to: /root/stanfordnlp_resources/en_ewt_models.zip
Extracting models file for: en_ewt
Cleaning up...Done.


# 1. Importing Files

In [5]:
queries_df = pd.read_csv('../input/ms-marco-queries/msmarco-doctrain-queries.tsv', sep = '\t', names = ['qid','query'])
queries_df = queries_df.set_index('qid')
display(queries_df.head(10))
display(queries_df.tail(10))
print(len(queries_df.index))

,query
qid,
1185869,)what was the immediate impact of the success ...
1185868,_________ justice is designed to repair the ha...
1183785,elegxo meaning
645590,what does physical medicine do
186154,feeding rice cereal how many times per day
457407,most dependable affordable cars
441383,lithophile definition
683408,what is a flail chest
484187,put yourself on child support in texas


,query
qid,
234427,how fast should a power chair go
285891,how many hours do you need for an aa
1164802,what causes dry cough?
87046,causes of irritated mouth tissue
562255,what are nephridia?
19285,anterolisthesis definition
558837,what are fishing flies
559149,what are fsh levels during perimenopause
706678,what is a yowie


367013


In [ ]:
print(os.listdir('./')) # This will print the content of current directory
print(os.listdir('../input')) # This will print the content of input directory

In [ ]:
# Install and import relevant libraries
!python -m easy_install ../input/compiledlucene/bk/lucene-8.1.1-py3.6-linux-x86_64.egg
!cp -r ../input/compiledlucene/bk/JCC-3.7-py3.6-linux-x86_64.egg /opt/conda/lib/python3.6/site-packages/
import sys
sys.path
sys.path.append('/opt/conda/lib/python3.6/site-packages/JCC-3.7-py3.6-linux-x86_64.egg')
sys.path.append('/opt/conda/lib/python3.6/site-packages/lucene-8.1.1-py3.6-linux-x86_64.egg')

In [ ]:
import lucene

# 2. EDA

In [6]:
# Let's first check for any null rows

null_rows = queries_df[queries_df.isnull().any(axis = 1)]
display(null_rows)

# Cool, there aren't any

,query
qid,


In [7]:
# Now let's try to find the average length of a query

def return_length(n):
    return len(n)

total_len = np.sum(queries_df['query'].apply(return_length))
average_query_length = total_len/len(queries_df)
print(total_len)
print(average_query_length)

# 33 characters seems reasonable.

12163703
33.14243092206543


In [8]:
# Average number of word counts in the query

def number_of_words(n):
    words = n.split(' ')
    return len(words)

total_word_count = np.sum(queries_df['query'].apply(number_of_words))
average_word_count = total_word_count/len(queries_df)
print(total_word_count)
print(average_word_count)

# An average word count of 6 seems about right for a query

2185955
5.956069676006027


In [9]:
# Let's now create a corpus of the 2.1M words used in the queries

def build_vocab(sentences, verbose =  True):
    vocab = {}
    for sentence in tqdm(sentences, disable = (not verbose)):
        for word in sentence.split(' '):
            try:
                vocab[word] += 1
            except KeyError:
                vocab[word] = 1
    return vocab

sentences = queries_df['query'].values
# print(sentences)
vocab = build_vocab(sentences)
print({k: vocab[k] for k in list(vocab)[:5]})

# As we can see, there are some tokens that are misspelled. I'll have to handle that later

100%|██████████| 367013/367013 [00:01<00:00, 272426.00it/s]

{')what': 1, 'was': 10493, 'the': 77087, 'immediate': 27, 'impact': 153}


In [10]:
# Let's now sort the vocab, that way we can remove the mispelled words

sorted_vocab = sorted(vocab.items(), key=lambda x: x[1], reverse=True)

# Let's first the see the top 10 most common words in the text, and their counts
print(sorted_vocab[:5])

# Let's now have a look at the last 50 terms in the sorted list (they will most probably be mispellings)
print(sorted_vocab[-50:])


# WOAH!! This is surprising, the last items of the sorted list, aren't actually mispelling, but many of them are just words ending with a question mark or bracket. I am gonna leave them be for now.

[('what', 147773), ('is', 133329), ('the', 77087), ('of', 59991), ('a', 55371)]
[('abbati', 1), ('ridgedale', 1), ('diffuses', 1), ('metts', 1), ('pedipalp', 1), ('synulox', 1), ('pureblood', 1), ('atd', 1), ('westheimer', 1), ('tears?', 1), ('creo', 1), ('air,', 1), ('bulbasaur', 1), ('squirtle', 1), ('thrilling', 1), ('omelettes', 1), ('inherited?', 1), ('(how', 1), ('it)', 1), ("parent's", 1), ('electrocuted', 1), ('impinging', 1), ('industry-leading', 1), ('abuja', 1), ('bicubic', 1), ('spenic', 1), ('ex-convicts', 1), ('electroforming', 1), ('buttress?', 1), ('sparely', 1), ('diabetics?', 1), ('448', 1), ('mooring', 1), ('250k', 1), ('exercise/walking', 1), ('pulmonologist?', 1), ('tympanitis', 1), ('rheostat?', 1), ('nauseousness', 1), ('meggitt', 1), ('f81', 1), ('intergenic', 1), ('detasseling', 1), ('julington', 1), ('ciss', 1), ('fantastical', 1), ('filezilla', 1), ('clarisse.', 1), ('nephridia?', 1), ('yowie', 1)]


# 3. Evaluation Metrics & Supporting Functions

In [11]:
def query_sampler(df, percentage_of_samples = 0.9):
    """ 
    This function creates a sample set of queries from the orignal dataset
    
    Args:
        df (dataframe) -> The original dataframe
        percentage_of_samples (float) -> Between 0 and 1
        
    Returns:
        Sampled set of queries.
    
    """
    l = len(df)
    number_of_samples = int(l*percentage_of_samples)
    print("Number of instances being sampled is", number_of_samples)
    randomList = random.sample(range(0, l), number_of_samples)
    return(df.iloc[randomList])


In [12]:
def break_the_query(query_string):
    """
    This function breaks the query down on character level.
    
    Args:
        query_string(str) -> Original query string
        
    Returns:
        list_of_strings(iterable) -> A list of multiple sub-strings made from the query
    """
    list_of_strings = []
    l = len(query_string)
    for i in range (1, l-1):
        s = query_string[:i]
        list_of_strings.append(s)
    return list_of_strings
    

In [13]:
def train_test_split(df, train_percentage = 0.9):
    """ 
    This function creates a sample set of queries from the orignal dataset
    
    Args:
        df (dataframe) -> The original dataframe
        train_percentage (float) -> Represents the percentage of training examples. Between 0 and 1
        
    Returns:
        Training and testing data
    
    """
    l = len(df)
    number_of_training_samples = int(l*train_percentage)
    print("Number of instances being sampled for training data is", number_of_training_samples)
    train_List = random.sample(range(0, l), number_of_training_samples)
    test_List = []
    for i in range(l):
        if i not in train_List:
            test_List.append(i)
    return(df.iloc[train_List], df.iloc[test_List])


In [14]:
def distance_score(str1, str2):
    """
    This function calculates the Levenshtein distance between the two strings. Will use the library 'editdistance' for this task.
    
    Args:
        str1(string) -> String 1
        str2(string) -> String 2
        
    Returns:
        d (float) -> Represents the leveshtein distance between the two strings
    """
    d = editdistance.eval(str1, str2)
    return d

def best_distance_on_query(model, query_string):
    """
    This function returns the best distance score from all the suggestions made for a query.
    
    Args:
        model (object) -> Trained model object 
        query_string (string) -> The query string
        
    Returns:
        Best Levenshtein distance score of all the suggestions made by the model.
    """
    query_suggestions = model_1.getAutoSuggestions(query_string)
    least_distance = len(query_string) * 5
    for query_suggestion in query_suggestions:
        distance_measure = distance_score(query_suggestion, query_string)
        if distance_measure < least_distance:
            least_distance = distance_measure
    return least_distance

# 4. AutoSuggest Methods

## Part 1: Dictionary Based Approach

#### Preparing Data
Ideally the dictionary-based approach (without analyzers) shouldn't include words in the build dictionary, but I am gonna try it out as well. 
So, the data would be prepared for TRIE in two batches:
1. Simple queries data.
2. Simple queries data, along with all the words used in that queries data.

In [ ]:
# This cell contains the point 1 of the two data forms described above.

#First I am gonna create a sample of ~36000 queries
queries_df_1 = query_sampler(queries_df, percentage_of_samples = 0.1)

# Next up, I'll remove the qid from the index
queries_df_1.reset_index(drop = True, inplace = True)

# Next I'll divide the queries data into train and test set. With 95% training data
train_queries_df_1, test_queries_df_1 = train_test_split(queries_df_1, train_percentage = 0.95)
train_queries_data_1 = train_queries_df_1['query'].values
test_queries_data_1 = test_queries_df_1['query'].values
print(train_queries_data_1)
print(len(train_queries_data_1))
#print(queries_data_1)
#print(len(queries_data_1))

In [ ]:
# This cell contains data created by definition of point 2 of the two data forms described above.

#First I am gonna create a sample of ~36000 queries
queries_df_2 = query_sampler(queries_df, percentage_of_samples = 0.1)

# Next up, I'll remove the qid from the index
queries_df_2.reset_index(drop = True, inplace = True)

# Next up I am gonna add all the words in these queries seperately as well
sentences = queries_df_2['query'].values
tokens = []
for sentence in tqdm(sentences):
    for word in sentence.split(' '):
        # I will also have to remove the special characters from the words before putting them into the vocab. I will not remove numerics because many search queries might require numbers like WC20 or XL5 etc
        token = ""
        for character in word:
            if character.isalnum():
                token += character
        tokens.append(token)
vocab = list(set(tokens))
for word in vocab:
    queries_df_2 = queries_df_2.append({'query' : word}, ignore_index=True)
    
display(queries_df_2.tail(10))
display(len(queries_df_2))

# Next I'll divide the queries data into train and test set. With 95% training data
train_queries_df_2, test_queries_df_2 = train_test_split(queries_df_2, train_percentage = 0.95)
train_queries_data_2 = train_queries_df_2['query'].values
test_queries_data_2 = test_queries_df_2['query'].values


#### Model Creation: TRIE

In [ ]:
class TrieNode(): 
    def __init__(self): 
          
        # Initialising one node for trie 
        self.children = {} 
        self.last = False
  

class Trie(): 
    def __init__(self): 
          
        # Initialising the trie structure. 
        self.root = TrieNode() 
        self.word_list = []
        self.count = 0
  
    def formTrie(self, keys): 
          
        # Forms a trie structure with the given set of strings 
        # if it does not exists already else it merges the key 
        # into it by extending the structure as required 
        for key in keys: 
            self.insert(key) # inserting one key to the trie. 
  
    def insert(self, key): 
          
        # Inserts a key into trie if it does not exist already. 
        # And if the key is a prefix of the trie node, just  
        # marks it as leaf node. 
        node = self.root 
  
        for a in list(key): 
            if not node.children.get(a): 
                node.children[a] = TrieNode() 
  
            node = node.children[a] 
  
        node.last = True
  
    def search(self, key): 
          
        # Searches the given key in trie for a full match 
        # and returns True on success else returns False. 
        node = self.root 
        found = True
  
        for a in list(key): 
            if not node.children.get(a): 
                found = False
                break
  
            node = node.children[a] 
  
        return node and node.last and found 
  
    def suggestionsRec(self, node, word): 
          
        # Method to recursively traverse the trie 
        # and return a whole word.  
        if node.last: 
            self.word_list.append(word) 
  
        for a,n in node.children.items(): 
            self.suggestionsRec(n, word + a)
            
    def limitedSuggestionsRec(self, node, word, no_of_suggestions): 
          
        # Method to recursively traverse the trie 
        # and return a whole word. But limit the number of routes ventured to no_of_suggestions
        if node.last:
            self.count = self.count + 1
            if self.count <= no_of_suggestions:
                self.word_list.append(word)

        for a,n in node.children.items():
            self.suggestionsRec(n, word + a) 
            
    
    def printAutoSuggestions(self, key, no_of_suggestions = 0): 
        # Returns all the words in the trie whose common 
        # prefix is the given key thus listing out all  
        # the suggestions for autocomplete. 
        node = self.root 
        not_found = False
        temp_word = '' 
        print('This function was called')
        characters = list(key)
        for a in characters:
            if not node.children.get(a): 
                not_found = True
                break
  
            temp_word += a 
            node = node.children[a] 
  
        if not_found: 
            return 0
        elif node.last and not node.children: 
            return -1
  
        if no_of_suggestions>0:
            self.count = 0
            self.limitedSuggestionsRec(node, temp_word, no_of_suggestions)
        else:
            self.suggestionsRec(node, temp_word)
  
        for s in self.word_list: 
            print(s) 
        return 1
    
    def getAutoSuggestions(self, key, no_of_suggestions = 0): 
          
        # Returns all the words in the trie whose common 
        # prefix is the given key thus listing out all  
        # the suggestions for autocomplete. 
        node = self.root 
        not_found = False
        temp_word = '' 
  
        characters = list(key)
        for a in characters:
            if not node.children.get(a): 
                not_found = True
                break
  
            temp_word += a 
            node = node.children[a] 
  
        if not_found: 
            return 0
        elif node.last and not node.children: 
            return -1
        
        if no_of_suggestions>0:
            self.count = 0
            self.limitedSuggestionsRec(node, temp_word, no_of_suggestions)
        else:
            self.suggestionsRec(node, temp_word)
   
        return self.word_list

#### Model Training
We are gonna create two TRIE structures (one for each kind of data)

In [ ]:
model_1 = Trie()
model_1.formTrie(train_queries_data_1)

In [ ]:
model_2 = Trie()
model_2.formTrie(train_queries_data_2)

#### Model Testing
We are gonna test each of the two models trained above using 3 metrics of evaluation:
1. Intuition Based (Will analyze the relevance of the search suggestions manually)
2. Distance Based (Will produce the overall score of test set using Levenshtein Distance)
3. Relevance Based (Will use semantic textual similarity to determine relevance of each of the suggestions).

##### Model 1:

In [ ]:
# I am first gonna define the three strings I'll be using for Intuition based testing
string1 = 'What building method might use a balloon frame?'
string2 = 'What is causing rash on arms'
string3 = 'What causes your glands on the top of your throat to swell'

# Next up, I am gonna create a list of sub strings from these queries
list_string1 = break_the_query(string1)
list_string2 = break_the_query(string2)
list_string3 = break_the_query(string3)

# Now, I'll print the query results after each new character being typed in
comp = model_1.printAutoSuggestions(string1) 
if comp == -1: 
    print("No other strings found with this prefix\n") 
elif comp == 0: 
    print("No string found with this prefix\n") 
break


## Part 2: Analyzer Based Approach

#### Data Preparation
I am gonna prepare the data with the following steps:
1. Tokenize the data
2. Create a Dictionary of Words, and their contexts.

In [15]:
#First I am gonna create a sample of ~36000 queries
queries_df_3 = query_sampler(queries_df, percentage_of_samples = 0.1)

# Next up, I'll remove the qid from the index
queries_df_3.reset_index(drop = True, inplace = True)
display(queries_df_3.head(10))

Number of instances being sampled is 36701


,query
0,what musician was known as slow hand
1,how much does a lawyer charge to help me file ...
2,how big is a playing card
3,why do echoes occur
4,how much education is need to become a pharmacist
5,health maintenance organization is what type o...
6,wier definition
7,what is adverse selection economics
8,irs id verification
9,gingivoplasty definition


In [16]:
# The problem with this data is that it cannot be used for NER tasks as it is, that is, it is not capitalized. And therefore, will not be able to identify NER tags properly.
# To resolve this problem, we will perform manual True Casing by using POS tagger of StanfordNLP, and then the results thus produced would be fed into SpaCy NER model.

stf_nlp = st.Pipeline(processors='tokenize,mwt,pos')
spacy_nlp = spacy.load('en_core_web_sm')

Use device: cpu
---
Loading: tokenize
With settings: 
{'model_path': '/root/stanfordnlp_resources/en_ewt_models/en_ewt_tokenizer.pt', 'lang': 'en', 'shorthand': 'en_ewt', 'mode': 'predict'}
---
Loading: pos
With settings: 
{'model_path': '/root/stanfordnlp_resources/en_ewt_models/en_ewt_tagger.pt', 'pretrain_path': '/root/stanfordnlp_resources/en_ewt_models/en_ewt.pretrain.pt', 'lang': 'en', 'shorthand': 'en_ewt', 'mode': 'predict'}
Done loading processors!
---


In [ ]:
class Analyzer:
    """
    This class performs several analysis on the query data (NER, POS, etc.) and creates a word dictionary of a form appropriate for usage in fast-autocomplete library
    
    Args:
        df (DataFrame) -> Contains all the queries
        
    Returns:
        words (dictionary) -> Dictionary of the fast-autocomplete prescribed format
    
    """
    def __init__(self, df):
        self.words = {}
        self.df = df
        
        
    def get_entities(self, query_string):
        '''
        This function takes in the query string and returns all the Named Entities present in the caseless string being passed to it.
        
        Args:
            query_string (string) -> A single query of the user
            
        Returns:
            ner_words (list) -> Contains all the identified NER terms in the text, along with their corresponding tag and start and end points.
        
        '''
        # Here, I'll first perform Truecasing using StanfordNLP, and then use NER model from Spacy
        doc = stf_nlp(query_string)
        truecased_list = [w.text.capitalize() if w.upos in ["PROPN","NNS"] else w.text for sent in doc.sentences for w in sent.words]
        truecased_query = ''
        for words in truecased_list:
            truecased_query.append(word)
            truecased_query.append(' ')
        
        
        
    def add_word_to_dictionary(self, current_word, prior1_word = None, prior2_word = None, ner_context = None):
        '''
        This function adds a new word/phraze to the words dictionary
        
        Args:
            current_word (string) -> A token (word/phraze) to be added to the dictionary
            ner_context (string) -> Describes the type of entity, if there is one
            prior1_word (string) -> The word immediately prior to the current one
            prior2_word (string) -> The word before the prior1_word
        
        '''
        self.words[current_word] = []              # Initializing the word in words dictionary
        self.words[current_word][0] = {}           # Initializing the Context dictionary for that word
        self.words[current_word][1] = current_word         # Setting the display value equal to the word string
        self.words[current_word][2] = 1            # Initializing the Count to be equal to zero
        if prior1_word != None:
            self.words[current_word][0]["prior1"] = prior1_word
        if prior2_word != None:
            self.words[current_word][0]["prior2"] = prior2_word
        # Write the code for adding the NER value to the context of each current_word
        if ner_context != None:
            self.words[current_word][0]["type"] = ner_context 
        # Write the code for adding a POS tag value to the context of each current_word. Will have to decide whether or not I want to do this.
     
    
    
    def get_words_dictionary(self, context_size = 1):
        """
        This function creates a dictionary of words in the format that can be directly fed into fast-autocomplete model.

        Args:
            context_size (int) -> The size of the context window to be turned into the dictionary format.
        """
        for i in range(len(self.df)):
            query = self.df['query'][i]
            ner_words = get_entities(query_string = query)
            query_words = query.split(' ')
            







# This function would create the dictionary of words along with their contexts

def create_words_dictionary(df, context_size = 1):
    """
    This function creates a dictionary of words in the format that can be directly fed into fast-autocomplete model.
    
    Args:
        df (DataFrame) -> Contains all the queries
        context_size (int) -> The size of the context window to be turned into the dictionary format.
    """
    words = {}
    
    for i in range(len(df)):
        query = df['query'][i]
        ner_words = get_entities(query)             #gets the entities from the NER model
        query_words = query.split(' ')
        for j in range(len(query_words)):
            current_word = query_words[j]
            if current_word not in words:
                words[current_word] = []              # Initializing the word in words dictionary
                words[current_word][0] = {}           # Initializing the Context dictionary for that word
                words[current_word][1] = current_word         # Setting the display value equal to the word string
                words[current_word][2] = 1            # Initializing the Count to be equal to zero
                if (j-1) >= 0:
                    words[current_word][0]["prior1"] = query_words[j-1]
                if (j-2) >= 0:
                    words[current_word][0]["prior2"] = query_words[j-2]
                # Write the code for adding the NER value to the context of each current_word
                
                
                # Write the code for adding a POS tag value to the context of each current_word. Will have to decide whether or not I want to do this.
                
            else:
                context_of_existing_current_word = words[current_word][0]
                existing_context_list = context_of_existing_current_word.keys()
                new_word_context = {}
                if (j-1) >= 0:
                    new_word_context["prior1"] = query_words[j-1]
                if (j-2) >= 0:
                    new_word_context["prior2"] = query_words[j-2]
                # Writ the code for adding NER value to "new_word_context"
                
                
                
                # Write the code for adding POS tags to the "new_word_context"
                
                
                # Following code checks whether the word has been used in the same context or not
                same = True
                for context in existing_context_list:
                    if context in new_word_context.keys():
                        if new_word_context[context] == context_of_existing_current_word[context]:
                            same = True
                        else:
                            same = False
                            break
                if same == True:
                    words[current_word][2] += 1           # Increasing the count of the word by one
                else:
                    
                
            
        

words = {}

for line in csv_gen:
    make = line['make']
    model = line['model']
    if make != model:
        local_words = [model, '{} {}'.format(make, model)]
        while local_words:
            word = local_words.pop()
            if word not in words:
                words[word] = {}
    if make not in words:
        words[make] = {}
return words

## Part 3: Content Based Approach